# ELECTRIC VEHICLE CHARGE SCHEDULING 
Objective: Minimize the cost incurred by the charging station.

In [ ]:
import numpy as np
import copy
import random

In [ ]:
def cal_E_g(h, EV, deltaT, n, P_PV, C_g, len): 
  dC_EVs = np.empty((0, 2), dtype=[('energy_cost', np.int64), ('index', np.int64)])
  dC_EVs = np.append(dC_EVs, np.array([tuple([0,0])], dtype=dC_EVs.dtype))
  dC_EVs = np.append(dC_EVs, np.array([tuple([1000000000000,n+1])], dtype=dC_EVs.dtype))

  C_EVs = np.empty((0, 2), dtype=[('dep_time', np.int64), ('index', np.int64)])

  for i in range(1, n+1, 1):
    if (h*deltaT <= EV[i]['T_fin'] and h*deltaT > EV[i]['T_init']):
      if (EV[i]['type'] == True and EV[i]['deltaE'] > 0):
        dC_EVs = np.append(dC_EVs, np.array([tuple([EV[i]['EPT'][h-1], i])], dtype=dC_EVs.dtype))
        EV[i]['EPT'][h] = EV[i]['EPT'][h-1]
      else:
        if (EV[i]['type'] == False):
          C_EVs = np.append(C_EVs, np.array([tuple([len+1, i])], dtype=C_EVs.dtype))
        else:
          C_EVs = np.append(C_EVs, np.array([tuple([EV[i]['T_fin'], i])], dtype=C_EVs.dtype))

  dC_EVs = np.sort(dC_EVs, order=['energy_cost'])
  C_EVs = np.sort(C_EVs, order=['dep_time'])[::-1]

  j = 0
  for i in range(1, n+1, 1):
    if (EV[i]['type'] == True):
      EV[i]['C_curr'] = -1*EV[i]['EPT'][h-1]*EV[i]['E_curr']

  E_PV_curr = P_PV[h]*deltaT
  E_g_curr = 0;

  for i in range(0, C_EVs.size, 1):
    C_EV = C_EVs[i]['index']

    while ((dC_EVs[j]['index'] == 0 and -EV[C_EV]['deltaE'] > E_PV_curr) or (dC_EVs[j]['index'] != 0 and dC_EVs[j]['index'] != (n+1) and -EV[C_EV]['deltaE'] > EV[dC_EVs[j]['index']]['deltaE'])):
      if (dC_EVs[j]['index'] == 0):
        EV[C_EV]['E_curr'] += -E_PV_curr
        EV[C_EV]['deltaE'] += E_PV_curr
        E_PV_curr = 0
      else:
        EV[C_EV]['E_curr'] += -EV[dC_EVs[j]['index']]['deltaE']
        if (EV[C_EV]['type'] == True):
          EV[C_EV]['C_curr'] += EV[dC_EVs[j]['index']]['EPT'][h-1]*EV[dC_EVs[j]['index']]['deltaE']
        EV[C_EV]['deltaE'] += EV[dC_EVs[j]['index']]['deltaE']
        EV[dC_EVs[j]['index']]['E_curr'] += EV[dC_EVs[j]['index']]['deltaE']
        EV[dC_EVs[j]['index']]['deltaE'] = 0
      j += 1

    if (dC_EVs[j]['index'] == 0):
      EV[C_EV]['E_curr'] += EV[C_EV]['deltaE']
      E_PV_curr += EV[C_EV]['deltaE']
      EV[C_EV]['deltaE'] = 0

    elif (dC_EVs[j]['index'] == n+1):
      EV[C_EV]['E_curr'] += EV[C_EV]['deltaE']
      if (EV[C_EV]['type'] == True):
        EV[C_EV]['C_curr'] += C_g[h]*(-1*EV[C_EV]['deltaE'])
      E_g_curr += -1*EV[C_EV]['deltaE']
      EV[C_EV]['deltaE'] = 0  
    else:
      EV[C_EV]['E_curr'] += EV[C_EV]['deltaE']
      if (EV[C_EV]['type'] == True):
        EV[C_EV]['C_curr'] += EV[dC_EVs[j]['index']]['EPT'][h-1]*(-1*EV[C_EV]['deltaE'])
    
      EV[dC_EVs[j]['index']]['E_curr'] += -1*EV[C_EV]['deltaE']
      EV[dC_EVs[j]['index']]['deltaE'] += EV[C_EV]['deltaE']    
      EV[C_EV]['deltaE'] = 0

  for i in range(1, n+1, 1):
    if (h*deltaT <= EV[i]['T_fin'] and h*deltaT > EV[i]['T_init']):
      if (EV[i]['type'] == True and EV[i]['deltaE'] > 0):
        EV[i]['E_curr'] += EV[i]['deltaE']
        EV[i]['deltaE'] = 0
  
  for i in range(1, n+1, 1):
    if (h*deltaT <= EV[i]['T_fin'] and h*deltaT > EV[i]['T_init']):
      if (EV[i]['type'] == True and EV[i]['deltaE_const'] < 0):
        EV[i]['EPT'][h] = EV[i]['C_curr']/((-1)*EV[i]['E_curr'])
    EV[i]['deltaE_const'] = 0

  return E_g_curr

In [ ]:
def find_deltaE_Ranges(EV, n, h, deltaT, LR):
  for i in range(1, n+1, 1):
    if (h*deltaT > EV[i]['T_fin'] or h*deltaT <= EV[i]['T_init']): 
      LR[i][0] = 0
      LR[i][1] = 0
    elif (EV[i]['type'] == False):
      LR[i][0] = max(EV[i]['MinPwLmt']*deltaT, EV[i]['E_fin'] - EV[i]['E_curr'])
      LR[i][1] = min(0, EV[i]['E_fin'] - EV[i]['E_curr'] - EV[i]['MinPwLmt']*(EV[i]['T_fin'] - h*deltaT))
    else:
      LR[i][0] = max(max(EV[i]['MaxCap']-EV[i]['E_curr'], EV[i]['MinPwLmt']*deltaT), EV[i]['E_fin'] - EV[i]['E_curr'] - EV[i]['MaxPwLmt']*(EV[i]['T_fin'] - h*deltaT))
      LR[i][1] = min(min(-1*EV[i]['E_curr'], EV[i]['MaxPwLmt']*deltaT), EV[i]['E_fin'] - EV[i]['E_curr'] - EV[i]['MinPwLmt']*(EV[i]['T_fin'] - h*deltaT))

In [ ]:
def find_All_deltaE_Combinations(LR, n, n_curr, deltaE_comb, comb):
  if (n_curr == n+1):
    deltaE_comb.append(comb)
  else:
    for deltaE in range(LR[n_curr][0], LR[n_curr][1]+1, 1):
      comb1 = comb.copy()
      comb1.append(deltaE)
      find_All_deltaE_Combinations(LR, n, n_curr+1, deltaE_comb, comb1)
  return

In [ ]:
def find_Min_Cost(len, tot_cost, n, h, EV, P_PV, C_g):
  if (h == len+1):
    return tot_cost
  else:
    LR = np.zeros(shape=(100, 2), dtype = np.int64)
    find_deltaE_Ranges(EV, n, h, deltaT, LR)
    # if (h == 2):
    #   print(LR)
    deltaE_comb = []
    comb = [0]
    find_All_deltaE_Combinations(LR, n, 1, deltaE_comb, comb)
    # if (h == 2):
    #   print(deltaE_comb)
    mini = 10000000000000
    E_pre = [0]
    C_pre = [0]
    EPT_pre = [0]
    for i in range(1, n+1, 1):
      E_pre.append(EV[i]['E_curr'])
    for i in range(1, n+1, 1):
      C_pre.append(EV[i]['C_curr'])
    for i in range(1, n+1, 1):
      EPT_pre.append(EV[i]['EPT'][h-1])
    for comb in deltaE_comb: 
      tot_cost_temp = tot_cost
      for i in range(1, n+1, 1):
        EV[i]['deltaE_const'] = EV[i]['deltaE'] = comb[i]
        EV[i]['E_curr'] = E_pre[i]
        EV[i]['C_curr'] = C_pre[i]
        EV[i]['EPT'][h-1] = EPT_pre[i]
      E_g_curr = cal_E_g(h, EV, deltaT, n, P_PV, C_g, len)
      tot_cost_temp += C_g[h]*E_g_curr
      mini = min(mini, find_Min_Cost(len, tot_cost_temp, n, h+1, EV, P_PV, C_g))
    return mini

In [ ]:
import fileinput

inp = []
with fileinput.input(files = ('sample_data/h_16.txt')) as f:
    for line in f:
        for word in line.split():
            inp.append(word)
inp_itr = 0

# of 100 EVs in 1000 time slots
EV_E_curr = np.zeros(shape=(100,1000), dtype = np.int64)

EV = np.zeros(100, 
       dtype=[('type', np.bool_), ('SOC_init', np.int64), ('SOC_fin', np.int64), ('MaxCap', np.int64), ('MaxPwLmt', np.int64), ('MinPwLmt', np.int64), ('E_init', np.int64), ('E_fin', np.int64), ('E_curr', np.int64), ('TI', np.int64), ('T_init', np.int64), ('T_fin', np.int64), ('deltaE', np.int64), ('deltaE_const', np.int64), ('C_curr', np.int64), ('EPT', np.int64, 1000)])

new_arr = np.zeros(1000, dtype = np.int64)
C_g = copy.deepcopy(new_arr)
P_PV = copy.deepcopy(new_arr)
E_g = copy.deepcopy(new_arr)

H = int(inp[inp_itr])
inp_itr += 1
print('Enter the length of the day: ' + str(H))

deltaT = int(inp[inp_itr])
inp_itr += 1
print('Enter the length of one time unit: ' + str(deltaT))
len = (int)(H//deltaT)  
H = len*deltaT;
i = 0

print('Enter the grid prices over the whole day (' + str(len) + ' space separated no.s): ')
for i in range(1,len+1,1):
  print(inp[inp_itr] + " ", end = '')
  C_g[i] = int(inp[inp_itr])
  inp_itr += 1

print('\nEnter the photo voltaic power generated over the whole day (' + str(len) + ' space separated no.s): ')
for i in range(1,len+1,1):
  print(inp[inp_itr] + " ", end = '')
  P_PV[i] = int(inp[inp_itr])
  inp_itr += 1

n = int(inp[inp_itr])
inp_itr += 1
print("\nEnter number of EVs: " + str(n))
for i in range(1, n+1, 1):
  print("\nEnter details of EV" + str(i) + " :- \n")
  ev_type = inp[inp_itr]
  inp_itr += 1
  print("Enter the type of EV (VV/GV): " + ev_type)
  if (ev_type == "GV"):
    EV[i]['type'] = False
  else:
    EV[i]['type'] = True
  
  EV[i]['SOC_init'] = int(inp[inp_itr])
  inp_itr += 1
  print("Enter initial SOC of the EV: " + str(EV[i]['SOC_init']))
  EV[i]['SOC_fin'] = int(inp[inp_itr])
  inp_itr += 1
  print("Enter final SOC of the EV: " + str(EV[i]['SOC_fin']))
  EV[i]['MaxCap'] = int(inp[inp_itr])
  inp_itr += 1
  print("Enter the max capacity of the EV: " + str(EV[i]['MaxCap']))

  EV[i]['MaxCap'] = -1*EV[i]['MaxCap']
  EV[i]['MinPwLmt'] = int(inp[inp_itr])
  inp_itr += 1
  print("Enter the min power limit of the EV [-ve]: " + str(EV[i]['MinPwLmt']))
  EV[i]['MaxPwLmt'] = int(inp[inp_itr])
  inp_itr += 1
  print("Enter the max power limit of the EV (0 for GV)[+ve]: " + str(EV[i]['MaxPwLmt']))
  EV[i]['TI'] = int(inp[inp_itr])
  inp_itr += 1
  print("Enter the time interval of stay of EV: " + str(EV[i]['TI']))
  EV[i]['E_fin'] = (EV[i]['SOC_fin']*EV[i]['MaxCap'])/100
  EV[i]['E_curr'] = EV[i]['E_init'] = (EV[i]['SOC_init']*EV[i]['MaxCap'])/100
  while (EV[i]['TI']*EV[i]['MinPwLmt'] > EV[i]['E_fin'] - EV[i]['E_init']):
    EV[i]['TI'] = int(input("Invalid time interval of stay of EV.\nEnter a larger time interval: "))   

  EV[i]['T_init'] = int(inp[inp_itr])
  inp_itr += 1
  print("Enter arrival time of the EV: " + str(EV[i]['T_init']))
  EV[i]['T_fin'] = EV[i]['T_init'] + EV[i]['TI']


Enter the length of the day: 16
Enter the length of one time unit: 1
Enter the grid prices over the whole day (16 space separated no.s): 
1 4 4 3 2 4 1 4 4 3 2 4 4 3 2 4 
Enter the photo voltaic power generated over the whole day (16 space separated no.s): 
1 2 3 4 5 3 1 2 3 4 5 3 3 4 5 3 
Enter number of EVs: 5

Enter details of EV1 :- 

Enter the type of EV (VV/GV): GV
Enter initial SOC of the EV: 20
Enter final SOC of the EV: 70
Enter the max capacity of the EV: 10
Enter the min power limit of the EV [-ve]: -4
Enter the max power limit of the EV (0 for GV)[+ve]: 0
Enter the time interval of stay of EV: 10
Enter arrival time of the EV: 0

Enter details of EV2 :- 

Enter the type of EV (VV/GV): GV
Enter initial SOC of the EV: 30
Enter final SOC of the EV: 60
Enter the max capacity of the EV: 10
Enter the min power limit of the EV [-ve]: -2
Enter the max power limit of the EV (0 for GV)[+ve]: 0
Enter the time interval of stay of EV: 7
Enter arrival time of the EV: 3

Enter details of E

In [ ]:
min_tot_cost = find_Min_Cost(len, 0, n, 0, EV, P_PV, C_g)

print("Min cost to the Charging Station using Brute force: " + str(min_tot_cost))